In [ ]:
import drjit as dr
import drjit.nn as nn
import imageio.v3 as iio

from drjit.opt import Adam, GradScaler
from drjit.auto.ad import (
    Texture2f,
    TensorXf,
    TensorXf16,
    Float16,
    Float32,
    Array2f,
    Array3f,
)

In [ ]:

ref = TensorXf(
    iio.imread(
        "https://rgl.s3.eu-central-1.amazonaws.com/media/uploads/wjakob/2024/06/wave-128.png"
    )
    / 256
)
tex = Texture2f(dr.mean(ref, axis=-1)[:, :, None])

Normalizing flows can be used to both sample from a learned distribution, but
also evaluate the probability density function for a given sample. This makes
them very useful in computer graphics, where both properties are often
required.

A normalizing flow is represented by an invertible function $f_\theta$. To
sample random variables $X$ from the learned distribution, we sample latent
variables $Z$ from a normal gaussian distribution $Z \sim p_Z = N(0, 1)$, and
apply the inverse flow $X = f^{-1}_\theta(Z)$.

We parameterize the normalizing flows with coupling and permutation layers
$f_{i;\theta}$, such that $X = f_{0;\theta} \circ f_{1;\theta} \circ \dots
f_{D;\theta} (Z)$. To train the network, we maximize the log sum of the
estimated probability of sampling the sample i.e. $max \sum \text{log}
p_{X;\theta}(X_i)$. To compute this probability, we can sum over the log
determinant of the layers, $p_{X;\theta}(X) = \text{log} \left\vert \text{det} {\partial z
\over \partial x} \right\vert_{\theta} + \text{log} p_{Z}(Z)$.

In [ ]:


class PermutationLayer(nn.Module): ...


class CouplingLayer(nn.Module):

    def __init__(self, n_layers: int = 3, n_activations: int = 64) -> None:
        super().__init__()

        sequential = []
        for i in range(n_layers - 1):
            sequential.append(nn.Linear(n_activations, n_activations))
            sequential.append(nn.ReLU())

        sequential.append(nn.Linear(n_activations, n_activations))

        self.net = nn.Sequential(*sequential)

    def inverse(self, z: nn.CoopVec) -> nn.CoopVec:
        r"""
        This function represents the inverse evaluation of the coupling layer,
        i.e. $X = f^{-1}_\theta(Z)$.
        """
        z: list = list(z)
        d = len(z) // 2

        id, z2 = z[:d, d:]

        p = list(self.net(nn.CoopVec(id)))
        a, mu = p[:d], p[d:]

        x2 = (z2 - mu) * dr.exp(-a)

        x = nn.CoopVec(id, x2)
        return x

    def forward(self, x: nn.CoopVec) -> tuple[nn.CoopVec, Float16]:
        r"""
        This function evaluates the foward flow $Z = f_\theta(X)$, as well as
        the log jacobian determinant.
        """

        x = list(x)
        d = len(x) // 2

        id, x2 = x[:d], x[d:]

        p = list(self.net(nn.CoopVec(id)))
        a, mu = p[:d], p[d:]